In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.dataset.dataset import TextDataset
import torch 
from torch import nn

/home/dzhunk/University/lillama/src/dataset/dataset.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
train_data = TextDataset(
    corpus_path = 'data/tiny_stories_txt/TinyStoriesV3-GPT4-train.txt',
    save_tokenizer_to = 'data/vocab/bpe',
    max_len = 300,
    vocab_size = 1000,
    sample=10
)
print('created train data with size:', len(train_data))
test_data = TextDataset(
    corpus_path = 'data/tiny_stories_txt/TinyStoriesV3-GPT4-valid.txt',
    save_tokenizer_to = 'data/vocab/bpe',
    max_len = 300,
    vocab_size = 1000,
    sample=10
)
print('created test data with size:', len(test_data))

0it [00:00, ?it/s]

created train data with size: 10


0it [00:00, ?it/s]

created test data with size: 10


In [4]:
from torch.utils.data import DataLoader
from src.model.language_model import LanguageModel
from src.dataset.dataset import Collator
from src.utils.trainer import CosineAnnealingWithWarmupLR

collate_fn = Collator(pad_value=train_data.pad_id)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4, collate_fn=collate_fn)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LanguageModel(
    embed_dim=128,
    vocab_size=train_data.vocab_size,
    max_len=train_data.max_len,
    pad_idx=train_data.pad_id,
    num_layers=2,
    num_heads=2,
    dropout=0,
    feedforward_dim=128
)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
# lr_scheduler = CosineAnnealingWithWarmupLR(optimizer=optimizer, warmup_steps=50, max_steps=200)

In [5]:
from src.utils.wandb_logger import WandbLogger
from src.utils.trainer import train

wdb = WandbLogger(
    project_name="little-lama",
    config={"loh": "loh"}
)
# пробую переобучиться на маленьком датасете
train(model, optimizer, criterion, train_loader, train_loader, 600, DEVICE, wdb, log_output = False)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: dzhunkoffski. Use `wandb login --relogin` to force relogin


100%|██████████| 1/1 [00:00<00:00,  3.54it/s]


итоговый ран из wandb